In [ ]:
import json
import os
from decimal import Decimal
from pprint import pprint

import pandas as pd

from config import Config

In [ ]:
# trade_type = 'usdt_futures'
trade_type = 'spot'
cfg_symbols = sorted(json.load(open(f'{Config.BINANCE_DATA_DIR}/exginfo/{trade_type}.json')).keys())
ss = os.listdir(f'{Config.BINANCE_DATA_DIR}/candle_parquet_fixed/{trade_type}/1h')
ss = [s.split('.')[0] for s in ss]
missing_symbols = sorted(set(ss) - set(cfg_symbols))
print(missing_symbols)

In [ ]:
def guess(p, col):
    df = pd.read_parquet(p)
    c = df[col]
    tick = Decimal('1')
    while True:
        abs_rel_err = (c.round() / c - 1).abs()
        if abs_rel_err.max() <= 1e-8:
            return str(tick)
            break
        tick /= 10
        c *= 10
    
d = dict()

for symbol in missing_symbols:
    p = f'{Config.BINANCE_DATA_DIR}/candle_parquet_fixed/{trade_type}/1h/{symbol}.pqt'
    d[symbol] = {
        "price_tick": guess(p, 'close'),
        "lot_size": guess(p, 'volume'),
        "min_notional_value": "5"
    }    

pprint(d)